 <font size="6">Обучение на реальных данных</font>

# Проблемы при работе с реальной задачей машинного обучения

С какими проблемами чаще всего сталкиваются люди, работающие с машинным обучением, в реальной жизни?
- нехватка данных
- недостаток размеченных данных
- не качественная разметка
- низкое качество изображений
- не сбалансированность датасета
- переобучение
- **какие еще?**


[How to avoid machine learning pitfalls: a guide for academic researchers (Lones, 2021)](https://arxiv.org/abs/2108.02497)

Если у вас недостаточно данных, то обучение хорошо обобщающей модели может оказаться невозможным. Выяснить, генерализуется модель или нет, может быть непросто, пока вы не начнете строить модели: все зависит от соотношения сигнал/шум в наборе данных.

Если сигнал сильный, то можно обойтись меньшим количеством данных; если слабый, то необходимо больше данных. Если вы не можете получить больше данных - а это распространенная проблема во многих исследовательских тогда вы можете лучше использовать имеющиеся данные, используя перекрестную проверку (*cross validation*). Вы также можете использовать методы увеличения данных (например, см. [Understanding data augmentation for classification: when to warp? (Wong et al., 2016)](https://arxiv.org/abs/1609.08764) и [A survey on Image Data Augmentation for Deep Learning (Shorten and Khoshgoftaar, 2019)](https://journalofbigdata.springeropen.com/articles/10.1186/s40537-019-0197-0)). Такие методы могут быть весьма эффективны для повышения эффективности небольших наборов данных. Увеличение данных также полезно в ситуациях, когда у вас ограниченное количество данных в определенных частях вашего набора данных, например, в задачах классификации, когда у вас меньше образцов в одних классах, чем в других - ситуация, известная как дисбаланс классов (см. обзор методов решения этой проблемы в [Learning from class-imbalanced data: Review of methods and applications (Haixiang et al., 2017)](https://www.sciencedirect.com/science/article/abs/pii/S0957417416307175?via%3Dihub)).

Однако если у вас ограниченное количество данных, то, скорее всего, вам также придется ограничить сложность используемых вами ML-моделей, поскольку модели с большим количеством параметров, такие как глубокие нейронные сети, могут легко переоценить маленькие наборы данных. В любом случае, важно выявить эту проблему на ранней стадии и выработать подходящую (и обоснованную) стратегию для ее решения.

# Как решить проблему малого количества данных

<!-- ResNet (популярная архитектура для классификации изображений), заняла 1-е место в классификационном конкурсе ILSVRC 2015 с улучшением примерно на 50% по сравнению с предыдущим уровнем развития техники. ResNet не только имела очень сложную и глубокую архитектуру, но и была обучена на 1,2 млн изображений.

Как в отрасли, так и в академических кругах хорошо известно, что при достаточно большом количестве данных, очень разные алгоритмы DL работают практически одинаково. Следует отметить, что большие данные должны содержать значимую информацию, а не просто шум, чтобы модель могла извлекать уроки из них. -->
[Блог-пост о том как решать проблему малого количества данных.](https://towardsdatascience.com/breaking-the-curse-of-small-datasets-in-machine-learning-part-1-36f28b0c044d)

<center><img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img_licence/l11_1-1.png" width="800"></center>
<center><em>На рисунке показаны основные проблемы, с которыми приходится сталкиваться при работе с небольшими наборами данных, а также возможные подходы и методы их решения.</em></center>


## Работа с несбалансированным датасетом

Сначала, давайте рассмотрим методы работы с датасетами в которых классы представлены не равномерно.


Сгенерируем датасет

In [ ]:
from sklearn.datasets import make_classification
import seaborn as sns
import matplotlib.pyplot as plt

X_data, Y_data = make_classification(n_samples=10000, n_features=10, n_classes=2)
sns.displot(Y_data, height=4)
plt.show()

И теперь его испортим (изменим соотношение класса `0` и класса `1`)

In [ ]:
import numpy as np

Y_spoiled = np.random.choice(Y_data[Y_data == 1], size=np.int(0.1 * len(Y_data[Y_data == 1])))
Y_spoiled = np.hstack([Y_data[Y_data == 0], Y_spoiled])
X_spoiled = X_data[Y_spoiled]

sns.displot(Y_spoiled, height=4)
plt.show()

### Что можно сделать, что бы как-то улучшить ситуацию используя методы ML

- Можно **изменить функцию потерь**: для задач классификации мы часто используем кросс-энтропийный лосс и редко используем среднюю абсолютную ошибку или среднеквадратичную ошибку для обучения и оптимизации нашей модели.

- В случае несбалансированных данных, модель учится распознавать доминирующий класс чаще чем все остальные, соответственно, она выучивает статистическое распределение классов и считает что чем чаще она предсказывает этот класс - тем меньше она ошибается. Что бы как-то решить эту проблему - мы можем **добавить веса к потерям**, соответствующим различным классам, чтобы выровнять это смещение данных. Например, если у нас есть два класса в соотношении 4:1, мы можем применить веса в соотношении 1:4 к вычислению функции потерь, чтобы данные были сбалансированы. 

Этот метод помогает нам легко смягчить проблему несбалансированных данных и улучшить обобщение модели для разных классов. Например:

In [ ]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(Y_spoiled), y=Y_spoiled)
print(f'class_weights: {class_weights}')

Мы также можем указать явные веса классов в соответствии с нашими требованиями, дополнительные сведения смотри в документации Scikit-learn.



```
from sklearn.linear_model import LogisticRegression

  # Create decision tree classifer object
 clf = LogisticRegression(random_state=0, class_weight='balanced')

  # Train model
 model = clf.fit(X_data, Y_spoiled)
```



Еще один способ решение проблемы несбалансированных данных - собственно их балансирование. Это может быть сделано либо за счет увеличения частоты класса меньшинства (_oversampling_), либо за счет уменьшения частоты класса большинства (_undersampling_) с помощью методов случайной или кластерной выборки.

Обычно _oversampling_ предпочтителен, когда общий размер данных небольшой, а _undersampling_ полезен, когда у нас есть большой объем данных. Точно так же случайная или кластерная выборка определяется тем, насколько хорошо распределены данные.


<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img_licence/l11_0.png" width="600">

Вот тут можно прочитать подробнее: [Imbalanced Data: How to handle Imbalanced Classification Problems](https://www.analyticsvidhya.com/blog/2017/03/imbalanced-data-classification/).

`Resampling` может быть легко выполнен с помощью пакета [imbalanced-learn](https://pypi.org/project/imbalanced-learn/), как показано ниже:

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
import warnings
warnings.filterwarnings('ignore')

rus = RandomUnderSampler(random_state=42)
X_rus, Y_rus = rus.fit_resample(X_spoiled, Y_spoiled)
sns.displot(Y_rus, height=3)
plt.show()

### Генерация синтетических данных

Хотя `oversampling` или `undersampling` помогает сбалансировать данные, дублирование данных увеличивает вероятность переобучения.

Другой подход к решению этой проблемы - создание синтетических данных с помощью данных о классе меньшинств.

**Synthetic Minority Over-sampling Technique (SMOTE)** или **Modified- SMOTE** - два таких метода, которые генерируют синтетические данные. Проще говоря, SMOTE берет точки данных класса меньшинства и создает новые точки данных, которые лежат между любыми двумя ближайшими точками данных, соединенными прямой линией.

Для этого алгоритм вычисляет расстояние между двумя точками данных в пространстве признаков, умножает расстояние на случайное число от 0 до 1 и помещает новую точку данных на этом новом расстоянии от одной из точек данных, используемых для определения расстояния. **Обратите внимание,** что количество ближайших соседей, учитываемых для генерации данных, также является гиперпараметром и может быть изменено в зависимости от требований.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img_vera/l11_0-1.png" width="700">

**M-SMOTE** - это модифицированная версия SMOTE, которая также принимает во внимание базовое распределение класса меньшинства.

Алгоритм классифицирует образцы классов меньшинств на 3 отдельные группы - образцы безопасности / безопасности, образцы границ и образцы скрытого шума.

Это делается путем вычисления расстояний между выборками класса меньшинства и выборками обучающих данных.

В отличие от SMOTE, алгоритм случайным образом выбирает точку данных из k ближайших соседей для выборки безопасности, выбирает ближайшего соседа из выборок границ и ничего не делает для устранения скрытого шума.

Вот тут можно прочитать подробнее: [SMOTE explained for noobs - Synthetic Minority Over-sampling TEchnique line by line](https://rikunert.com/SMOTE_explained)

Создадим датасет и посмотрим на то какие в нем данные

In [ ]:
from collections import Counter

# define dataset
X_data, Y_data = make_classification(n_samples=10000, n_features=2, n_redundant=0,
	n_clusters_per_class=1, weights=[0.99], flip_y=0, random_state=42)
# summarize class distribution
counter = Counter(Y_data)
print(f'0 class: {counter[0]}')
print(f'1 class: {counter[1]}')

In [ ]:
from numpy import where

# scatter plot of examples by class label
for label, _ in counter.items():
	row_ix = where(Y_data == label)[0]
	plt.scatter(X_data[row_ix, 0], X_data[row_ix, 1], label=str(label))
plt.legend()
plt.show()

А теперь давайте сделаем oversampling класас меньшинства с помощью SMOTE и построим график преобразованного набора данных.

Мы будем использовать реализацию `SMOTE` из библиотеки `imbalanced-learn`.

In [ ]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_data, Y_data = oversample.fit_resample(X_data, Y_data)
# summarize the new class distribution
counter = Counter(Y_data)
print(f'0 class: {counter[0]}')
print(f'1 class: {counter[1]}')
# scatter plot of examples by class label
for label, _ in counter.items():
	row_ix = where(Y_data == label)[0]
	plt.scatter(X_data[row_ix, 0], X_data[row_ix, 1], label=str(label))
plt.legend()
plt.show()

### Обнаружение аномалий / изменений

В случае сильно несбалансированных наборов данных, таких как мошенничество или машинный сбой, стоит задуматься, могут ли такие примеры рассматриваться как аномалия (выброс) или нет. Если такое событие и впрямь может считаться аномальным, мы можем использовать такие модели, как `OneClassSVM`, методы кластеризации или методы обнаружения гауссовских аномалий.

Эти методы требуют изменения способа мышления: мы будем рассматривать аномалии, как отдельный класс выбросов. Это может помочь нам найти новые способы разделения и классификации.

Обнаружение изменений похоже на обнаружение аномалий, за исключением того, что мы ищем изменение или отличие, а не аномалию. Это могут быть изменения в поведении пользователя, наблюдаемые по шаблонам использования или банковским транзакциям.

Разберем пример обнаружения аномалий с помощью `OneClassSVM` из библиотеки Sk-Learn (там же, можно найти еще множеество различных алгоритмов)

In [ ]:
from sklearn.datasets import make_blobs

# Create dataset
X_data, _ = make_blobs(n_samples=1000, centers=1, cluster_std=1.1, center_box=(0, 0))
plt.scatter(X_data[:, 0], X_data[:, 1], alpha=0.25)
plt.show()

Настроим наш детектор аномалий

In [ ]:
from sklearn.svm import OneClassSVM

svm = OneClassSVM(kernel="rbf", gamma=0.01, nu=0.03)
print(f'SVM parameters:\n{svm}')
svm.fit(X_data)
pred = svm.predict(X_data)

Продемонстрируем аномальные точки предсказанные детектором

In [ ]:
anom_index = where(pred == -1)
values = X_data[anom_index]

plt.scatter(X_data[:, 0], X_data[:, 1], alpha=0.25)
plt.scatter(values[:, 0], values[:, 1], color="r", marker="x")
plt.show()

### Методы объединения (Ensembling Techniques)

Помимо того, что бы делать что-то с данными, мы можем попробовать сделать что-то с самой моделью. 
Идея объединения нескольких слабых учеников / разных моделей показала отличные результаты при работе с несбалансированными наборами данных.

`Bagging` и `Boosting` методы также показали отличные результаты при решении различных задач, и их следует изучить вместе с методами, описанными выше, для получения лучших результатов.



**Вывод:**

Мы рассмотрели несколько наиболее часто используемых методов работы с несбалансированными данными для традиционных алгоритмов машинного обучения.

Один или многие из вышеперечисленных методов могут быть хорошей отправной точкой в ​​зависимости от конкретной задачи.

### 8 тактик борьбы с несбалансированными классами в наборе данных машинного обучения

[Блог пост про 8 тактик борьбы с несбалансированными классами в наборе данных машинного обучения](https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/)

1. **Определите, можете ли вы собрать больше данных?**
2. **Попробуйте изменить метрику оценки модели.**
    Accuracy не является показателем, который следует использовать при работе с несбалансированным набором данных. [Есть метрики, которые были разработаны для работы с несбалансированными классами.](https://machinelearningmastery.com/classification-accuracy-is-not-enough-more-performance-measures-you-can-use/#:~:text=Classification%20Accuracy%20is%20Not%20Enough%3A%20More%20Performance%20Measures%20You%20Can%20Use,-By%20Jason%20Brownlee&text=When%20you%20build%20a%20model,This%20is%20the%20classification%20accuracy)

3. **Применить `Resampling`** (см. выше)
4. **Попробовать создать синтетические образцы данных**
5. **Не следует использовать свой любимый алгоритм для каждой задачи.**
    Необходимо проверять различные типы алгоритмов для решения конкретной проблемы. Например, `Random Forest` часто хорошо работает с несбалансированными наборами данных.
6. **Попробуйте модели с санкциями**
    Штрафная классификация накладывает дополнительные штрафы на модель за ошибки классификации в классе меньшинства во время обучения. Эти штрафы могут склонить модель к уделению большего внимания классу меньшинства.
7. **Попробуйте другую точку зрения**
    Есть области исследований, посвященные несбалансированным наборам данных. Двe, которые вы могли бы рассмотреть, - это [обнаружение аномалий](https://en.wikipedia.org/wiki/Anomaly_detection) и [обнаружение изменений](https://en.wikipedia.org/wiki/Change_detection).
8. **[Попробуйте проявить творческий подход](https://www.quora.com/In-classification-how-do-you-handle-an-unbalanced-training-set)**.
    По-настоящему залезьте внутрь своей проблемы и подумайте, как разбить ее на более мелкие проблемы, которые легче решить.

### Не используйте точность для несбалансированных наборов данных

Будьте осторожны с тем, какие метрики вы используете для оценки ваших ML-моделей. Например, в случае моделей классификации, наиболее часто используемой метрикой является точность (*accuracy*), которая представляет собой долю сэмплов в наборе данных, которые были правильно классифицированы моделью.

Этот метод хорошо работает, если классы сбалансированы, т.е. каждый класс представлен одинаковым количество образцов в наборе данных. Но многие наборы данных не являются сбалансированными, и в этом случае точность может быть очень обманчивой метрикой. Рассмотрим, например, датасет, в котором 90% сэмплов представляют один класс, а 10% сэмплов представляют
другой класс. Бинарный классификатор, который всегда выдает первый класс, независимо от его входных данных, будет иметь точность 90%, несмотря на то, что он совершенно бесполезен. В такой ситуации предпочтительнее использовать такие метрики, как коэффициент каппы Коэна или коэффициент корреляции Мэтьюса (MCC), которые относительно нечувствительны к дисбалансу размеров классов.


# Аугментация
Другой способ побороть маленькое количество данных для обучения - аугментация. Сам термин пришел из музыки:

**Аугмента́ция** (позднелат. augmentatio — увеличение, расширение) — [техника ритмической композиции в старинной музыке](https://ru.wikipedia.org/wiki/Аугментация).


Модели глубокого обучения обычно требуют большого количества данных для обучения. В целом, чем больше данных, тем лучше для обучения модели. В то же время получение огромных объемов данных сопряжено со своими проблемами (например с нехваткой размеченных данных или с трудозатратами сопряженными с разметкой).

Вместо того, чтобы тратить дни на сбор данных вручную, мы можем использовать методы аугментации для автоматической генерации новых примеров из уже имеющихся.

Помимо увеличения размеченных датасетов, многие методы *self-supervised learning* построены на использовании разных аугмеентаций одного и того же сэмпла.


<center><img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img_vera/l11_2.png" width="700"></center>
<center><em>Примеры аугментаций картинки. </em></center>

Для аугментации будем использовать мультимодальную (работающую с различными модальностями, как например текст, картинки, звук и тд) библиотеку с забавным названием `AugLy` (*созвучно с ugly - стремный*).

In [ ]:
# ATTENTION, AFTER INSTALLATION, AN ERROR MESSAGE WILL APPEAR 
# COLAB WILL RESTART. IGNORE AND CONTINUE EXECUTION 

!pip install -U augly
!sudo apt-get install python3-magic
import os
os.kill(os.getpid(), 9)

In [ ]:
import torch
import random
import numpy as np
# fix random_seed
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# Compute on cpu or gpu 
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

**Важный момент**: при обучении модели мы используем разбиение данных на `train-val-test`. Аугментации стоит применять только на `train`. Почему так? Конечная цель обучения нейросети - это применение на реальных данных, которые сеть не видела. Вот и портить их не надо.

В любом случае, `test` должен быть отделен от данных еще до того как они попали в `DataLoader` или нейросеть.

Другое дело, что аугментации на тесте можно использовать как метод ансамблинга в случае классификации. Можно взять sample -> создать несколько его копий -> по разному их аугментировать -> предсказать класс на каждой из этих аугментированных копий -> а потом выбрать наиболее вероятный класс голосованием (такой функционал реализован например в [YOLOv5](https://github.com/ultralytics/yolov5/blob/d204a61834d0f6b2e73c1f43facf32fbadb6b284/models/yolo.py#L121), о которой речь пойдет в следующих лекциях).

## Изображения

Импортируем библиотеку и отобразим пример картинки. Картинку отмасштабируем, что бы она не занимала весь экран

In [ ]:
import os
import augly.image as imaugs
import augly.utils as utils
from IPython.display import display

# Get input image, scale it down to avoid taking up the whole screen
input_img_path = os.path.join(
    utils.TEST_URI, "image", "inputs", "dfdc_1.jpg"
)

# We can use the AugLy scale augmentation
input_img = imaugs.scale(input_img_path, factor=0.2)
display(input_img)

Рассмотрим несколько примеров аугментаций картинок. С полным списком можно ознакомиться на [сайте библиотеки](https://github.com/facebookresearch/AugLy/blob/main/augly/image/__init__.py).

### Rotation

Посмотрим какие параметры принимает на вход `Random Rotation`

In [ ]:
? imaugs.RandomRotation

Создадим перменную `transform` в которую добавим нашу аугментацию и применим ее к исходному изображению. Затем запустим следующую ячейку несколько раз подряд

In [ ]:
transform = imaugs.RandomRotation(min_degrees=-45.0, # Minimum threshold value of rotation
                                  max_degrees=45     # Maximum threshold value of rotation
                                  )

aug_image = transform(input_img)
display(aug_image)

Обратите внимание как `AugLy` делает `crop` - на выходе мы получаем только ту часть повернутого изображения, которая содержит собственно изображение

### Blur

In [ ]:
? imaugs.RandomBlur

In [ ]:
transform = imaugs.RandomBlur(min_radius=0.1, # Minimum threshold value of blur  (more large radius, make more blur of image)
                              max_radius=10   # Maximum threshold value of blur
                              )
aug_image = transform(input_img)
display(aug_image)

### Aspect Ratio

In [ ]:
? imaugs.RandomAspectRatio

In [ ]:
transform = imaugs.RandomAspectRatio(min_ratio=0.1, 
                                     max_ratio=3
                                     ) 
aug_image = transform(input_img)
display(aug_image)

### Совмещаем несколько аугментаций вместе

Для этого будем использовать метод `Compose`. Нам нужно будет создать `list` со всеми аугментациями, которые будут применены последовательно.

In [ ]:
transform = imaugs.Compose(
    [
     imaugs.RandomPixelization(min_ratio=0.1,
                               max_ratio=0.9),
     imaugs.RandomEmojiOverlay(emoji_size=0.25, 
                               x_pos=0.43, 
                               y_pos=0.2),
    ]
)

aug_image = transform(input_img)
display(aug_image)

### А что если мы хотим применять аугментации случайным образом?

Для этого воспользуемся методом `ApplyLambda`, который на вход принимает метод аугментации, и вероятность `p` что эта аугментация будет применена.

In [ ]:
? imaugs.ApplyLambda

В этот раз мы добавим еще один элемент случайности - будем генерировать цвет фона и цвет текста случайным образом.

In [ ]:
meme_bg_color=(np.random.randint(0,255), # Red channel - random values 0 to 255
               np.random.randint(0,255), # Green channel 
               np.random.randint(0,255)) # Blue channel

text_color = (255-meme_bg_color[0], # Find a contrasting color
              255-meme_bg_color[1], 
              255-meme_bg_color[2])

transform_to_apply = imaugs.MemeFormat(caption_height=50, 
                                       meme_bg_color=meme_bg_color,
                                       text_color=text_color)

transform = imaugs.ApplyLambda(aug_function=transform_to_apply, p=0.9)

aug_image = transform(input_img)
display(aug_image)

### Аугментация внутри `Dataset`

Возьмем папку с картинками

In [ ]:
image_folder = os.path.join(utils.TEST_URI, "image", "inputs")
input_img_path = os.listdir(image_folder)
input_img_path = [image_folder +'/' + x for x in input_img_path]
print(f'Num of images: {len(input_img_path)}')

Напишем класс `Dataset`

In [ ]:
from torch.utils.data import Dataset

class AugmentationDataset(Dataset):
    def __init__(self, imgs_list, transforms=None):
        self.imgs_list = imgs_list
        self.transforms = transforms

    def __len__(self):
        return len(self.imgs_list)

    def __getitem__(self, i):
        img = imaugs.scale(self.imgs_list[i], factor=0.5)
        if self.transforms is not None:
            img = self.transforms(img)
        return torch.tensor(img, dtype=torch.float)

Напишем вспомогательную функцию для отображения картинок

In [ ]:
def show_img(img):
    plt.figure(figsize=(10, 10))
    img_np = img.numpy()
    plt.imshow(np.transpose(img_np, (1, 2, 0)))
    plt.axis('off')
    plt.show()

Создадим `list` с аугментациями, которые мы хотим применить

In [ ]:
augmentations = [
    imaugs.RandomBrightness(),
    imaugs.RandomEmojiOverlay(),
    imaugs.RandomRotation(),
    imaugs.Resize(128,128),
]

`AugLy` кросс-фреймворковая библиотека, которая работает с `PIL` картинками. Что бы загрузить аугментации в `PyTorch`, нам необходимо эти картинки преобразовать в тензоры. Для этого воспользуемся родным торчевым преобразованием `transforms.ToTensor()`

In [ ]:
import torchvision.transforms as transforms
tensor_transforms = transforms.Compose(augmentations + [transforms.ToTensor()])

Теперь обернем все в `DataLoader` и отобразим

In [ ]:
from torch.utils.data import DataLoader
import torch, torchvision
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

Augmentation_dataloader = DataLoader(
    AugmentationDataset(input_img_path, tensor_transforms), batch_size=3
)

data = iter(Augmentation_dataloader)
show_img(torchvision.utils.make_grid(data.next()))

## Аудио

Рассмотрим несколько примеров аугментаций аудио. С полным списком можно ознакомиться на [сайте библиотеки](https://github.com/facebookresearch/AugLy/blob/main/augly/image/__init__.py).

Импортируем библиотеку и посмотрим на пример

In [ ]:
from IPython.display import display, Audio

# Get input audio
input_audio ='https://edunet.kea.su/repo/EduNet-web_dependencies/L11/audio_example.mp3'

display(Audio(input_audio))

Загрузим наш аудиоофайл в Colab

In [ ]:
from augly.audio.utils import validate_and_load_audio
import warnings
warnings.filterwarnings('ignore')

input_audio_arr, sr = validate_and_load_audio(input_audio) # sr - sampling rate

### Background Noise

In [ ]:
? audaugs.AddBackgroundNoise

In [ ]:
import numpy as np
import augly.audio as audaugs

snr = np.random.randint(1,10) # Random Signal-to-Noise ratio

transform = audaugs.AddBackgroundNoise(snr_level_db=snr) 
aug_audio, aug_sr = transform(input_audio_arr, sample_rate=sr)

display(Audio(aug_audio, rate=aug_sr))

Сравним волновые картины и спектрограммы

In [ ]:
import matplotlib.pyplot as plt
from scipy.signal import spectrogram

def produce_plots(input_audio_arr, aug_audio, sr):
    f,t, Sxx_in = spectrogram(input_audio_arr, fs=sr) # Compute spectrogram for the original signal (f - frequency, t - time)
    f,t, Sxx_aug = spectrogram(aug_audio, fs=sr)

    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(20,5)) 

    ax[0,0].plot(input_audio_arr)
    ax[0,0].set_xlim(0,len(input_audio_arr))
    ax[0,0].set_xticks([])
    ax[0,0].set_title('Original audio')

    ax[0,1].plot(aug_audio)
    ax[0,1].set_xlim(0,len(input_audio_arr))
    ax[0,1].set_xticks([])
    ax[0,1].set_title('Augmented  audio')

    ax[1,0].imshow(np.log(Sxx_in), 
                   extent = [t.min(), t.max(), f.min(), f.max()],
                   aspect='auto',
                   cmap='inferno')
    ax[1,0].set_ylabel('Frequecny, Hz')
    ax[1,0].set_xlabel('Time,s')
    
    ax[1,1].imshow(np.log(Sxx_aug), 
                   extent = [t.min(), t.max(), f.min(), f.max()],
                   aspect='auto',
                   cmap='inferno')
    ax[1,1].set_ylabel('Frequecny, Hz')
    ax[1,1].set_xlabel('Time,s')

    plt.subplots_adjust(hspace=0)
    plt.show()

produce_plots(input_audio_arr, aug_audio, sr)

### Time Stretch

In [ ]:
? audaugs.TimeStretch

In [ ]:
stretch_factor = np.random.uniform(0.25, 0.75) # Random stretching coefficient

transform = audaugs.TimeStretch(rate=stretch_factor) 
aug_audio, aug_sr = transform(input_audio_arr, sample_rate=sr)

display(Audio(aug_audio, rate=aug_sr))

### Совмещаем несколько аугментаций вместе

Как и в случае с картинками мы можем совмещать несколько аугментаций вместе

In [ ]:
? audaugs.ChangeVolume()

In [ ]:
time_between_clicks = np.random.uniform(0.5, 1)

transform = audaugs.Compose(
    [
     audaugs.Clicks(seconds_between_clicks=time_between_clicks), # Add clicks
     audaugs.Reverb(), # и add reverb
    ]
)

aug_audio, aug_sr = transform(input_audio_arr, sample_rate=sr)

display(Audio(aug_audio, rate=aug_sr))

Посмотрим на то, что получилось

In [ ]:
produce_plots(input_audio_arr, np.mean(aug_audio, axis=0), sr)

Обратите внимание, что порядок имеет значение

In [ ]:
time_between_clicks = np.random.uniform(0.5, 1)

transform = audaugs.Compose(
    [
     audaugs.Reverb(), # add reverb first 
     audaugs.Clicks(seconds_between_clicks=time_between_clicks), # then add clicks 
    ]
)

aug_audio, aug_sr = transform(input_audio_arr, sample_rate=sr)

display(Audio(aug_audio, rate=aug_sr))

In [ ]:
produce_plots(input_audio_arr, np.mean(aug_audio, axis=0), sr)

## Текст

Теперь рассмотрим несколько примеров аугментаций текста. С полным списком можно ознакомиться на [сайте библиотеки](https://github.com/facebookresearch/AugLy/blob/main/augly/text/__init__.py).

In [ ]:
import augly.text as textaugs

# Define input text
input_text = "Hello, future of AI for Science! How are you today?"
print(f'input text: {input_text}')

### Иммитация опечаток

In [ ]:
? textaugs.SimulateTypos()

In [ ]:
transform = textaugs.SimulateTypos()

aug_text = transform(input_text)
print(f'original  text: {input_text}')
print(f'augmented text: {aug_text}')

### Замена букв на похожие символы

In [ ]:
? textaugs.ReplaceSimilarChars

In [ ]:
transform = textaugs.ReplaceSimilarChars()

aug_text = transform(input_text)
print(f'original  text: {input_text}')
print(f'augmented text: {aug_text}')

И многое многое другое. А еще AugLy умеет работать с видео, но это выходит за рамки сегодняшнего обсуждения.

Кроме методов, реализованных в AugLy, существуют и специализированные библиотеки. 

Для аугментации изображений:
- [torchvision](https://pytorch.org/vision/stable/transforms.html)
- [Albumentations](https://albumentations.ai)
- [imgaug](https://imgaug.readthedocs.io/en/latest/index.html)


Для аугментации звука (и волновых функций в целом):
- [torchaudio](https://pytorch.org/tutorials/beginner/audio_preprocessing_tutorial.html)
- [torch-audiomentations](https://github.com/asteroid-team/torch-audiomentations)

Для аугментации текста:
- [TextAugment](https://github.com/dsfsi/textaugment)

**При выборе методов аугментации имеет смысл использовать те, которые будут в реальной жизни. Например, нет смысла делать перевод изображения в черно-белое, если предполагается, что весь входящий поток будет цветным, или отражать человека вверх-ногами, если мы не предполагаем его таким распознавать.**

### $\color{brown}{\text{Дополнительная информация}}$ 

###### Примеры аугументации в [TORCHVISION.TRANSFORMS](https://pytorch.org/vision/stable/transforms.html)

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_13.png" width="700">

In [ ]:
import os
from zipfile import ZipFile
from IPython.display import clear_output

os.chdir('/content')
# download files
!wget --no-check-certificate 'https://edunet.kea.su/repo/EduNet-web_dependencies/L11/for_transforms.Compose.zip' -O data.zip
with ZipFile('data.zip', 'r') as folder: # Create a ZipFile Object and load sample.zip in it
    folder.extractall() # Extract all the contents of zip file in current directory
clear_output()

In [ ]:
os.chdir("/content/for_transforms.Compose")
img_list = os.listdir()
print(img_list)

In [ ]:
from torch.utils.data import Dataset

class AugmentationDataset(Dataset):
    def __init__(self, img_list, transforms=None):
        self.img_list = img_list
        self.transforms = transforms

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, i):
        img = plt.imread(self.img_list[i])
        img = Image.fromarray(img).convert("RGB")
        img = np.array(img).astype(np.uint8)

        if self.transforms is not None:
            img = self.transforms(img)
        return torch.tensor(img, dtype=torch.float)


def show_img(img):
    plt.figure(figsize=(40, 38))
    img_np = img.numpy()
    plt.imshow(np.transpose(img_np, (1, 2, 0)))
    plt.show()

Существует несколько методов аугментации изображений, и мы обсудим некоторые из наиболее распространенных и широко используемых.

Вы можете попробовать другие методы в соответствии с вашими требованиями.

###### Image Rotation

In [ ]:
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
import torchvision
from PIL import Image
import numpy as np
import warnings
warnings.filterwarnings("ignore")

torch.manual_seed(42)
np.random.seed(42)

transform = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize((164, 164)),
        transforms.RandomRotation(50, expand=True),
        transforms.Resize((164, 164)),
        transforms.ToTensor(),
    ]
)

Augmentation_dataloader = DataLoader(
    AugmentationDataset(img_list, transform), batch_size=8, shuffle=True
)

data = iter(Augmentation_dataloader)
show_img(torchvision.utils.make_grid(data.next()))

###### Random Cropping

In [ ]:
transform = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize((164, 164)),
        transforms.RandomCrop((120, 120)),
        transforms.ToTensor(),
    ]
)

Augmentation_dataloader = DataLoader(
    AugmentationDataset(img_list, transform), batch_size=8, shuffle=True
)

data = iter(Augmentation_dataloader)
show_img(torchvision.utils.make_grid(data.next()))

###### Random Perspective

In [ ]:
transform = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize((164, 164)),
        transforms.RandomPerspective(distortion_scale=0.5, p=1.0),
        transforms.ToTensor(),
    ]
)

Augmentation_dataloader = DataLoader(
    AugmentationDataset(img_list, transform), batch_size=8, shuffle=True
)

data = iter(Augmentation_dataloader)
show_img(torchvision.utils.make_grid(data.next()))

###### Gaussian Blur

In [ ]:
transform = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize((164, 164)),
        transforms.GaussianBlur(7, sigma=(0.1, 2.0)),
        transforms.ToTensor(),
    ]
)

Augmentation_dataloader = DataLoader(
    AugmentationDataset(img_list, transform), batch_size=8, shuffle=True
)

data = iter(Augmentation_dataloader)
show_img(torchvision.utils.make_grid(data.next()))

###### Random Erasing

In [ ]:
transform = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize((164, 164)),
        transforms.ToTensor(),
        transforms.RandomErasing(),
    ]
)

Augmentation_dataloader = DataLoader(
    AugmentationDataset(img_list, transform), batch_size=8, shuffle=True
)

data = iter(Augmentation_dataloader)
show_img(torchvision.utils.make_grid(data.next()))

Существует и более сложные способы аугментации:

- **Mixup**

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img_vera/l11_3.png" width="700">

- **Аугументация при помощи синтеза данных**

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img_vera/l11_4.png" width="600">

В домашнем задании Вам предстоит провести эксперимент: обучить нейронную сеть на датасете без аугментации, и после нее.



Кроме методов, реализованных в Pytorch, существуют и специализированные библиотеки для аугментации изображений. 

Например:
- [lbumentations](https://albumentations.ai)
- [imgaug](https://imgaug.readthedocs.io/en/latest/index.html)
- [augly](https://github.com/facebookresearch/AugLy)

**При выборе методов аугментации имеет смысл использовать те, которые будут в реальной жизни. Например, нет смысла делать перевод изображения в черно-белое, если предполагается, что весь входящий поток будет цветным, или отражать человека вверх-ногами, если мы не предполагаем его таким распознавать.**

# Transfer Learning
----
Как быстро обучить нейросеть на своих данных, когда их мало?

Для таких типовых задач, как классификация изображений, можно воспользоваться готовой архитектурой (AlexNet, VGG, Inception, ResNet и т.д.) и обучить нейросеть на своих данных. Реализации таких сетей с помощью различных фреймворков уже существуют, так что на данном этапе можно использовать одну из них как черный ящик, не вникая глубоко в принцип её работы. Например в PyTorch есть много предобученных сетей: [`torchvision.models`](https://pytorch.org/vision/stable/models.html).


Однако, глубокие нейронные сети требовательны к большим объемам данных для сходимости обучения. И зачастую, в нашей частной задаче недостаточно данных для того, чтобы хорошо натренировать все слои нейросети. `Transfer Learning` решает эту проблему. Зачем обучать сеть заново, если можно использовать уже обученную на миллионе изображений и дообучить на свой датасет?

Для этого,  нужно отключить какие-то промежуточные слои. Тогда можно использовать то, что называется `Fine turning` - не нужно обучать всю модель, а достаточно только ее новую часть.

Нейронные сети, которые используются для классификации, как правило, содержат N выходных нейронов в последнем слое, где N — это количество классов. Такой выходной вектор трактуется как набор вероятностей принадлежности к классу. 

В реальных задачах распознавания изображений, количество классов может отличаться от того, которое было в исходном датасете. В таком случае нам придётся полностью выкинуть этот последний слой и поставить новый, с нужным количеством выходных нейронов

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img_vera/l11_5.png" width="700">

Зачастую в конце классификационных сетей используется полносвязный слой. Так как мы заменили этот слой, использовать предобученные веса для него уже не получится. Придется тренировать его с нуля, инициализировав его веса случайными значениями. Веса для всех остальных слоев мы загружаем из предобученной модели.

Существуют различные стратегии дообучения модели. Мы воспользуемся следующей: будем тренировать всю сеть из конца в конец (end-to-end), а предобученные веса не будем фиксировать, чтобы дать им немного скорректироваться и подстроиться под наши данные. Такой процесс называется тонкой настройкой (fine-tuning).

## Структурные компоненты

Для решения задачи нам понадобятся следующие компоненты:

1. Описание модели нейросети
2. Пайплайн обучения
3. Инференс пайплайн
4. Предобученные веса для этой модели
5. Данные для обучения и валидации


<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img_licence/l11_6.png" width="750">

## Как обучать модель используя Transfer Learning

Код обучения модели состоит из следующих шагов:

1. Построение train/validation пайплайнов данных
2. Построение train/validation графов (сетей)
3. Надстраивание классификационной функция потерь (cross entropy loss) поверх train графа
4. Код, необходимый для вычисления точности предсказания на валидационной выборке во время обучения
5. Логика загрузки предобученных весов из снэпшота
6. Создание различных структур для обучения
7. Непосредственно сам цикл обучения (итерационная оптимизация)

Последний слой графа конструируется с нужным нам количеством нейронов и исключается из списка параметров, загружаемых из предобученного снэпшота.


## Практический пример Transfer Learning

Давайте рассмотрим пример практической реализации такого подхода ([код переработан из этой статьи](https://learnopencv.com/image-classification-using-transfer-learning-in-pytorch/)).

 Загрузим датасет и удалим из него 90% файлов



In [ ]:
import os
from IPython.display import clear_output

os.chdir('/content')
path = '/content/2750/'

!wget https://edunet.kea.su/repo/EduNet-web_dependencies/L11/EuroSAT.zip # http://madm.dfki.de/files/sentinel/EuroSAT.zip
!unzip EuroSAT.zip

from random import sample
for folder in os.listdir(path):
  files = os.listdir(path+folder)
  for file in sample(files,int(len(files)*0.9)):
      os.remove(path+folder+'/'+file)
      
clear_output()

Определим аугментации. Для разнообразия будем использовать родные аугментации из библиотеки PyTorch Vision

In [ ]:
import torch
import random
import numpy as np
from torchvision import transforms
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# Applying Transforms to the Data
img_transforms = {
    "train": transforms.Compose(
        [
            transforms.Resize(size=128),
            transforms.RandomRotation(degrees=15),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.ToTensor(),
        ]
    ),
    # No augmentations on valid data!
    "valid": transforms.Compose( 
        [
            transforms.Resize(size=128),
            transforms.ToTensor(),
        ]
    ),
    # No augmentations on test data!
    "test": transforms.Compose(
        [
            transforms.Resize(size=128),
            transforms.ToTensor(),
        ]
    ),
}

Создадим `datasets`

In [ ]:
import torch
from torchvision import datasets
import torchvision.utils

dataset = datasets.ImageFolder(root=path)
# split to train/valid/test
train_set, valid_set, test_set = torch.utils.data.random_split(dataset, [int(len(dataset)*0.8), int(len(dataset)*0.1), int(len(dataset)*0.1)])
# define augmentations
train_set.dataset.transform = img_transforms['train']
valid_set.dataset.transform = img_transforms['valid']
test_set.dataset.transform = img_transforms['test']

print(f'Train size: {len(train_set)}')
print(f'Valid size: {len(valid_set)}')
print(f'Test size: {len(test_set)}')

In [ ]:
from torch.utils.data import DataLoader
# Batch size
batch_size = 64

# Number of classes
num_classes = len(dataset.classes)

# Get a mapping of the indices to the class names, in order to see the output classes of the test images.
idx_to_class = {v: k for k, v in dataset.class_to_idx.items()}

# Size of Data, to be used for calculating Average Loss and Accuracy
train_data_size, valid_data_size = len(train_set), len(valid_set)

# Create iterators for the Data loaded using DataLoader module
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
print('indexes to class: ')
idx_to_class

У нас не так уж и много изображений!! Обычная нейронка справиться не должна.

Загрузим Alexnet без весов и попробуем обучить с нуля

In [ ]:
from torchvision import models
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

alexnet = models.alexnet(pretrained=False)
print(alexnet)

Затем мы заменяем последний слой модели Alexnet одним последовательным слоем с 4096 нейронами и который имеет `num_classes` выходов, соответствующих числу классов в нашем подмножестве.

То есть мы "сказали" нашей модели распознавать не 1000, а только N классов.

In [ ]:
# Change the final layer of AlexNet Model for Transfer Learning
import torch.nn as nn

alexnet.classifier[6] = nn.Linear(4096, num_classes) # change out classes, from 1000 to 10
alexnet.classifier.add_module("7", nn.LogSoftmax(dim=1)) # add activation
print(alexnet)

Затем мы определяем функцию потерь и оптимизатор, который будет использоваться для обучения.

Мы используем функцию Negative Loss Likelihood, поскольку она полезна для классификации нескольких классов и оптимизатор Adam - один из самых популярных оптимизаторов (например потому что он может адаптировать скорость обучения для каждого параметра индивидуально.

In [ ]:
import torch.optim as optim

# Define Optimizer and Loss Function
criterion = nn.NLLLoss()
optimizer = optim.Adam(alexnet.parameters(), lr=3e-4)
print(optimizer)

Для тренировки и валидации нашей можели напишем отдельную функцию, которую итмеет смысл изучить подробнее.

In [ ]:
import time

def train_and_validate(model, criterion, optimizer, num_epochs=25):
    """
    Function to train and validate
    Parameters
        :param model: Model to train and validate
        :param criterion: Loss Criterion to minimize
        :param optimizer: Optimizer for computing gradients
        :param epochs: Number of epochs (default=25)

    Returns
        model: Trained Model with best validation accuracy
        history: (dict object): Having training loss, accuracy and validation loss, accuracy
    """

    start = time.time()
    history = []
    best_acc = 0.0

    for epoch in range(num_epochs):
        epoch_start = time.time()
        print("Epoch: {}/{}".format(epoch + 1, num_epochs))

        # Set to training mode
        model.train()

        # Loss and Accuracy within the epoch
        train_loss = 0.0
        train_acc = 0.0

        valid_loss = 0.0
        valid_acc = 0.0

        train_correct = 0
        for i, (inputs, labels) in enumerate(train_loader):

            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()  # Clean existing gradients
            outputs = model(inputs)  # Forward pass - compute outputs on input data using the model
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagate the gradients
            optimizer.step()  # Update the parameters

            # Compute the total loss for the batch and add it to train_loss
            train_loss += loss.item() * inputs.size(0)
            # Compute correct predictions
            train_correct += (torch.argmax(outputs, dim=-1)== labels).float().sum()

        # Compute the mean train accuracy
        train_accuracy = 100 * train_correct / (len(train_loader)*batch_size)

        val_correct = 0
        # Validation - No gradient tracking needed
        with torch.no_grad():

            model.eval()  # Set to evaluation mode

            # Validation loop
            for j, (inputs, labels) in enumerate(valid_loader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)  # Forward pass - compute outputs on input data using the model
                loss = criterion(outputs, labels)  # Compute loss
                valid_loss += loss.item() * inputs.size(0)  # Compute the total loss for the batch and add it to valid_loss

                val_correct += (torch.argmax(outputs, dim=-1) == labels).float().sum()

        # Compute mean val accuracy       
        val_accuracy = 100 * val_correct / (len(valid_loader)*batch_size)

        # Find average training loss and training accuracy
        avg_train_loss = train_loss / (len(train_loader)*batch_size)

        # Find average training loss and training accuracy
        avg_valid_loss = valid_loss / (len(valid_loader)*batch_size)

        history.append([avg_train_loss, avg_valid_loss, train_accuracy, val_accuracy])

        epoch_end = time.time()

        print(
            "Epoch : {:03d}, Training: Loss: {:.4f}, Accuracy: {:.4f}%, \n\t\tValidation : Loss : {:.4f}, Accuracy: {:.4f}%, Time: {:.4f}s".format(
                epoch + 1,
                avg_train_loss,
                train_accuracy.detach().cpu(),
                avg_valid_loss,
                val_accuracy.detach().cpu(),
                epoch_end - epoch_start,
            )
        )



    return model, history

Теперь обучим нашу модель

In [ ]:
import warnings
warnings.filterwarnings('ignore')

num_epochs = 20
trained_model, history = train_and_validate(
    alexnet.to(device), criterion, optimizer, num_epochs
)

torch.save(history, "history_fresh.pt")

Посмотрим на графики

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

fig, ax = plt.subplots(ncols=2, figsize=(10,5))

history = np.array(history)
ax[0].plot(history[:,:2])
ax[0].legend(["Train Loss", "Val Loss"])
ax[1].plot(history[:,2:])
ax[1].legend(["Train Accuracy", "Val Accuracy"])
ax[0].set_xlabel("Epoch Number")
ax[1].set_xlabel("Epoch Number")
ax[0].set_ylabel("Loss")
ax[1].set_ylabel("Accuracy")
plt.savefig("loss_curve.png")
ax[0].grid()
ax[1].grid()
plt.show()

Ну так себе результаты

Теперь давайте попробовать использовать *Fine-tunning*. Загрузим предобученную Alexnet:

In [ ]:
del alexnet
alexnet = models.alexnet(pretrained=True)

Поскольку большинство параметров в нашей предварительно обученной модели уже обучены, мы сбрасываем значение поля requires_grad в значение false.

In [ ]:
# Freeze model parameters
for param in alexnet.parameters():
    param.requires_grad = False

**Заморозка параметров**: отключение переменных предобученной модели для последующих изменений. "Замораживая" переменные предобученной модели мы гарантируем, что обучаться будет только последний классификационный слой, а значения предобученной модели останутся прежними.

Теперь мы поменяем последний слой таким образом, что бы он вместо 1000 классов ImageNet нам выдавал количество классов которое у нас есть

In [ ]:
# Change the final layer of AlexNet Model for Transfer Learning
alexnet.classifier[6] = nn.Linear(4096, num_classes)
alexnet.classifier.add_module("7", nn.LogSoftmax(dim=1))

In [ ]:
# Define Optimizer and Loss Function
criterion = nn.NLLLoss()
optimizer = optim.Adam(alexnet.parameters(), lr=3e-4)

In [ ]:
num_epochs = 20
trained_model, history = train_and_validate(
    alexnet.to(device), criterion, optimizer, num_epochs
)

torch.save(history, "history_finetuning.pt")

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10,5))

history = np.array(history)
ax[0].plot(history[:,:2])
ax[0].legend(["Train Loss", "Val Loss"])
ax[1].plot(history[:,2:])
ax[1].legend(["Train Accuracy", "Val Accuracy"])
ax[0].set_xlabel("Epoch Number")
ax[1].set_xlabel("Epoch Number")
ax[0].set_ylabel("Loss")
ax[1].set_ylabel("Accuracy")
plt.savefig("loss_curve.png")
ax[0].grid()
ax[1].grid()
plt.show()

И сравним между собой

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10,5))

history_fresh = np.array(torch.load('history_fresh.pt'))
history_finetuning = np.array(torch.load('history_finetuning.pt'))

ax[0].plot(history_fresh[:,:2])
ax[0].plot(history_finetuning[:,:2])
ax[0].legend(["Train Loss", "Val Loss"])

ax[1].plot(history_fresh[:,2:])
ax[1].plot(history_finetuning[:,2:])
ax[1].legend(["Train Accuracy", "Val Accuracy"])
ax[0].set_xlabel("Epoch Number")
ax[1].set_xlabel("Epoch Number")
ax[0].set_ylabel("Loss")
ax[1].set_ylabel("Accuracy")
plt.savefig("loss_curve.png")
ax[0].grid()
ax[1].grid()
plt.show()

Определенно стало лучше =)

Посмотрим на картинки

In [ ]:
def predict(model, test_img_name, device):
    """
    Function to predict the class of a single test image
    Parameters
        :param model: Model to test
        :param test_img_name: Test image

    """

    transform = img_transforms["test"]
    test_img = torch.tensor(np.asarray(test_img_name)) 
    test_img = transforms.ToPILImage()(test_img)
    plt.imshow(test_img)

    test_img_tensor = test_img_name.unsqueeze(0).to(device)

    with torch.no_grad():
        model.eval()
        # Model outputs log probabilities
        out = model(test_img_tensor).to(device)
        ps = torch.exp(out).to(device)
        topk, topclass = ps.topk(3, dim=1)
        for i in range(3):
            print(
                "Predcition",
                i + 1,
                ":",
                idx_to_class[topclass.cpu().numpy()[0][i]],
                ", Score: ",
                round(topk.cpu().numpy()[0][i], 2),
            )

In [ ]:
print("Shoud be %s\n" % idx_to_class[0])
predict(
    trained_model.to(device),
    valid_set[[np.where([x[1] == 0 for x in valid_set])[0]][0][1]][0],
    device,
)

In [ ]:
print("Shoud be %s\n" % idx_to_class[6])
predict(
    trained_model,
    valid_set[[np.where([x[1] == 6 for x in valid_set])[0]][0][10]][0],
    device,
)

In [ ]:
print("Shoud be %s\n" % idx_to_class[8])
predict(
    trained_model,
    valid_set[[np.where([x[1] == 8 for x in valid_set])[0]][0][5]][0],
    device,
)

* Мы только что увидели, как использовать предварительно обученную модель, обученную для 1000 классов ImageNet.

* Она очень эффективно классифицирует изображения, принадлежащие к N различным классам, которые нас интересуют.

* Также мы убедились в эффективности такого подхода для обучении классификации на небольшом наборе данных.

# Few/One - Shot learning

Существуют задачи где не представляется возможным разбить данные на классы так что бы в каждом классе было достаточно много объектов.

Рассмотрим например задачу распознавания лиц.

<img src ="http://edunet.kea.su/repo/src/L01_Intro/img/mp/video.png"  width="700">

На вход системе подается фото лица человека. Требуется сопоставить его с другим изображениям(и) например хранящимися в БД и таким образом идентифицировать человека на фотографии.

На первый взгляд кажется что это задача классификации.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img_vera/l11_7.png" width="700">

Все изображения одного человека будем считать относящимися к одному классу и модель будет этот класс предсказывать. 

Для небольшой организации, в которой всего несколько десятков сотрудников такой подход может сработать.
При этом возникнут проблемы:

1. Чтобы обучить такую ​​систему, нам сначала потребуется много (сотни) разных 
изображений каждого сотрудника.

2. Когда человек присоединяется к организации или покидает ее придется поменять структуру модели и обучать ее заново.


Это практически невозможно, для крупных организаций, где набор и увольнение происходит почти каждую неделю. И в принципе невозможно для города масштаба Москвы или Лондона в котором миллионы жителей и сотни тысяч приезжих.

## Генерация embedding

Поэтому используется другой подход.
Вместо того что бы классифицировать изображения, модель учится выделять ключевые признаки и на их основе строить компактный вектор точно описывающий лицо.


<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/emb.png" width="500">


В англоязычной литературе такие вектора признаков называются **embedding** и мы тоже будем использовать это обозначение.





Может возникнуть вопрос "Не потеряем ли мы важную информацию сжав так сильно". 
Что бы ответить на него вспомним про фотороботы. 

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/photo_robot.png" width="700" >




Для получения фотореалистичного изображения лица достаточно нескольких ключевых признаков: Глаза, волосы рот, нос ...

Каждый из которых кодируется максимум несколькими сотнаями значений.

То есть вектора - признака из 128 значений будет более чем достаточно.
Правда интерпретировать значения которые закодирует в него нейросеть будет не столь просто.


Если нам удасться обучить модель кодировать в embedding признаки важные для сравнения, то мы сможем сравнивать веткора между собой.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/face_dist.png" width="500" >


При этом расстояние между векторами для лиц которые похожи друг на друга будут маленькими а у непохожих наборот большими.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/clustrobo.png" width="700" >

Это позволит найти человека сравнивая embedding его фотографии с другим из БД используя [KNN](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) или иной метод кластеризации.




Теперь, чтобы сеть могла обнаружить его лицо, нам требуется только одно изображение его лица, которое будет сохранено в базе данных. Используя его в качестве эталонного изображения, можно вычислять сходство для любого нового экземпляра,  представленного ей.

## Сиамская сеть (Siamese Network)

Как будет устроенна модель для генерации таких векторов-признаков?

Можно было бы использовать обычную сеть обученную для задачи классифиикации, и затем удалить из нее последний(е) слой.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/vanilla_gen.png" width="700">

Активации последнего слоя представляют собой отклики на некие высокоуровневые признаки важные для классификации и их можно интерпретировать как embeddind.

Такой подход будет работать, но можно заметно улучшить точность используя лосс-функцию котороя оценивает именно качество сравнения а не классификации.





Подход, который мы будем рассматривать, является реализацией методологии, описанной в статье [Siamese Neural Networks for One-shot Image Recognition (Koch et al., 2015)](https://www.cs.cmu.edu/~rsalakhu/papers/oneshot1.pdf).

Состоит он в следующем:

<center><img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img_vera/l11_8.png" width="700"></center>
<center><em>Используются две копии одной и той же сети, отсюда и название Siamese Networks.</em></center>

Два входных изображения ($x_1$ и $x_2$) проходят через одну и ту же сверточную сеть, на выходе для каждого изображения генерируется вектор признаков фиксированной длины $h_{x_{1}}$ и $h_{x_{2}}$.

Если модель обучена правильно, мы можем предположить: если два входных изображения принадлежат одному и тому же персонажу, то их векторы признаков также должны быть похожими, а если два входных изображения принадлежат разным символам, то их векторы признаков также будут разными.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img_licence/l11_9.png" width="700">

Таким образом, оцениевая расстояние между двумя векторами признаков 
которое будет мамым для похожих объектов и большим для различных мы сможем оценить их сходство.

Это центральная идея сиамских сетей.

## Triplet Loss

Какой loss использовать для обучения такой сети?

Видимо loss функция должна будет анализировать не один выход а два или больше.

Популярной на сегодняшний день является  `Triplet loss`, которой требуется  три embedding вместо двух. 

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img_licence/l11_10.png" width="600">

Что бы сгенерировать три эмбеддинга модель должна получать на вход три изображения. 

Первые два должны относится к одному и тому же объекту (человеку) а третье к другому.


Таким образом Триплет состоит из базового ("якорного" `anchor`), положительного и отрицательного образцов.

Сама лосс функция будет выглядеть следующим образом:

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img_licence/l11_10.png" width="600">


$\color{red}{\text{Перерисовать!  Обе верхних фотографии должны принадлежать одному объекту а не классу!
}}$


$$TripletLoss = \sum_{1}^{N} L_i(x_i^{a},x_i^{p},x_i^{n})$$

$$L_i(x_i^{a},x_i^{p},x_i^{n})=max(0,\left\| f(x_i^{a}) -f(x_i^{p}) \right\|_2^{2} - \left\| f(x_i^{a}) -f(x_i^{n}) \right\|_2^{2} + \alpha)$$

Где:


$x_i^{a}$ - базовое изображение (anchor)

$x_i^{p}$ - изображение того же объекта (positive)

$x_i^{n}$ - изображение другого объекта (negative)

$f(x)$ - выход модели (embedding) для входа $x$


$\left\| a \right\|_2$ -это L2 (Euclidean norm), соответственно $\left\| a \right\|_2^{2}$ это L2 в квадрате.









В `Triplet loss` расстояние между анкором и положительной выборки минимизировано, а расстояние между анкором и отрицательной выборки максимально.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img_licence/l11_11.png" width="600">

Но можно также использовать и `Contrastive Loss` о ней подробнее в статье [Dimensionality Reduction by Learning an Invariant Mapping (Hadsell et al., 2005)](http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf)

Такая модель не учится классифицировать изображение напрямую по какому-либо из выходных классов. Скорее, он изучает функцию подобия, которая принимает два изображения в качестве входных данных и выражает, насколько они похожи.

Как это решает две проблемы, о которых мы говорили выше?
- Для обучения этой сети нам не требуется слишком много экземпляров класса, а для построения хорошей модели достаточно лишь нескольких экземпляров.
- Но самое большое преимущество в простоте ее обучения в случае появления нового сотрудника.

### Реализация сиамской сети
---
на примере датасета [Georgia Tech face database](http://www.anefian.com/research/face_reco.htm)

### Загрузка данных

Загрузим небольшой фрагмент датасета с лицами. Внутри архива фото лиц сгруппированны по папкам



*   s1/
**  photo1.pgm
**  photo2.pgm
**  ...
*   s2/
*   s2/
*    ...
*   sn/

В каждой папке фото лица одного и того же человека.

In [ ]:
from IPython.display import clear_output
!wget http://edunet.kea.su/repo/src/L11_Transfer_learning/small_face_dataset.zip
!unzip small_face_dataset.zip
clear_output()

Что бы результаты воспроизводились зафиксируем SEED

In [ ]:
import os
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)


set_random_seed(42)

### Dataset for TripletLoss

Для TripletLoss потребуются три изображения: anchor, positive, negative и метод __get_item__ должен возвращать их нам. Первые два должны принадлежать одному человеку, а третье другому. 



In [ ]:
from torch.utils.data import Dataset
from glob import glob
import random 
from PIL import Image

class SiameseNetworkDataset(Dataset):
    
    def __init__(self,dir=None,transform=None, splitter = '/'):
        self.dir = dir
        self.splitter = splitter
        self.transform = transform        
        self.files = glob(f"{self.dir}/**/*.pgm",recursive=True)
        self.data =self.build_index()        
    
    def build_index(self):
      index = {}      
      for f in self.files:
        id = self.path2id(f) 
        if not id in index:
          index[id] = []
        index[id].append(f)
      return index
    
    def path2id(self,path):
        return path.replace(self.dir,"").split(self.splitter)[0]

    def __getitem__(self,index):
        anchor_path = self.files[index]
        positive_path = self.find_positive(anchor_path)
        negative_path = self.find_negative(anchor_path)
        
        # Loading the images
        anchor = Image.open(anchor_path)
        positive = Image.open(positive_path)
        negative = Image.open(negative_path)
                
        if self.transform is not None:  # Apply image transformations           
            anchor = self.transform(anchor)
            positive = self.transform(positive)
            negative = self.transform(negative)

        return anchor, positive , negative

    def find_positive(self,path):
        id = self.path2id(path) 
        all_exept_my = self.data[id].copy()
        all_exept_my.remove(path)
        return random.choice(all_exept_my)

    def find_negative(self,path):
        all_exept_my_ids = list(self.data.keys())
        id = self.path2id(path) 
        all_exept_my_ids.remove(id)
        selected_id = random.choice(all_exept_my_ids)
        return random.choice(self.data[selected_id])
    
    def __len__(self):
        return len(self.files)

Выведем несколько изображений что бы убедится что класс датасета функционирует должным образом.

In [ ]:
from torch.utils.data import  DataLoader
import torch
import torchvision
import matplotlib.pyplot as plt
import torchvision.transforms as transforms

# Create dataset instance
siamese_dataset = SiameseNetworkDataset("faces/training/",
                                          transform=transforms.Compose([
                                            transforms.Resize((105,105)),
                                            transforms.ToTensor(),
                                          ])
                                        )

# Create dataloader & extract batch of data from it
vis_dataloader = DataLoader(siamese_dataset, batch_size =8, shuffle=True)
dataiter = iter(vis_dataloader)
example_batch = next(dataiter) # anc, pos, neg

# Show batch contents 
concatenated = torch.cat((example_batch[0],example_batch[1],example_batch[2]),0)
grid = torchvision.utils.make_grid(concatenated)

plt.axis("off")
plt.imshow(grid.permute(1,2,0).numpy())
plt.gcf().set_size_inches(20,60)
plt.show()    

В каждом столбце тройка изображений.Первое и второе принадлежат одному человеку, третье - другому.

### Создание модели

Нас устроит любая модель для работы с изображениями. Например Resnet18. 

Все что от нас требуется это:
- заменить последний слой
- отправлять на анализ три изображения вместо одного. Соответственно на выходе тоже будут три вектора признаков (embedding)


Пожалуй единственный вопрос это размерность последнего слоя . В промышленных системмах распознавания лиц, которые тренируются на датасетах из миллионов изображений, используются embedding размерностью от 128 до 512.

Значит для демонстрационной задачи нам с запасом хватить 64 значений. Значит  выход последнего линейного слоя должен быть равен 64.

In [ ]:
import torch
from torch import nn
from torchvision.models import resnet18

class SiameseNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = resnet18(pretrained = False)
        # Because we use grayscale images reduce input channel count to one
        self.model.conv1 = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        # Replace ImageNet 1000 class classifier with 64- out linear layer 
        self.model.fc = nn.Linear(self.model.fc.in_features, 64)

    def forward(self, anchor, positive, negative):
        output1 = self.model(anchor)
        output2 = self.model(positive)
        output3 = self.model(negative)
        
        return output1, output2, output3

### Dataloaders

Загрузчики данных не отличаются от загрузчиков для обычной сети. 
Единственное отличие это две аугментации которые добавили к данным:

*   Случайный поворот по вертикали (RandomHorizontalFlip)
*   Размытие (GaussianBlur)


In [ ]:
from torchvision import transforms as img_transf
from torchvision import datasets as ds
from torch.utils.data import DataLoader

# Apply augmentations on train data
img_trans_train = img_transf.Compose(
    [
        img_transf.Resize((105, 105)),
        img_transf.RandomHorizontalFlip(),
        img_transf.GaussianBlur(3, sigma=(0.1, 2.0)),
        img_transf.ToTensor(),
    ]
)

img_trans_test = img_transf.Compose(
    [img_transf.Resize((105, 105)), img_transf.ToTensor()]
)

train_dataset = SiameseNetworkDataset("faces/training/",transform = img_trans_train)
val_dataset = SiameseNetworkDataset("faces/testing/",transform = img_trans_test)


train_loader = DataLoader(train_dataset, num_workers=2, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, num_workers=2, batch_size=1, shuffle=False) 



### Обучение

Отличие от  сетей для классификации в том что у модели 3 выхода и все их надо передать в loss. При этом нет меток в явном виде.
Какой embedding отностся к позитивному образцу а какой к негативному определяется только порядком их следования.

In [ ]:
def train(num_epochs, model, criterion, optimizer, train_loader):
    loss_history = []
    l = []
    model.train()
    for epoch in range(0, num_epochs):
        
        for i, batch in enumerate(train_loader, 0):
            anc, pos, neg = batch
            output_anc, output_pos, output_neg = model(anc.to(device), pos.to(device), neg.to(device))
            loss = criterion(output_anc, output_pos, output_neg)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            l.append(loss.item())
        last_epoch_loss =  torch.tensor(l[-len(train_loader):-1]).mean()
        print("Epoch {} with {:.4f} loss".format(epoch,last_epoch_loss))
        
    return l, last_epoch_loss

При использовании GPU, обучение на 5-ти эпохах займет около 15 сек:

In [ ]:
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = SiameseNet().to(device)
criterion = nn.TripletMarginLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001 ) 

l, _ = train(5, model, criterion, optimizer, train_loader)

Выведем график loss

In [ ]:
import matplotlib.pyplot as plt

plt.plot([i for i in range(len(l))], l)
plt.ylabel('loss')
plt.xlabel('num of epochs')
plt.grid()
plt.show()

Видно что он график лосс уже вышел на плато, значит можно проверять результат.

### Проверка

Для начала выведем тройки изображений из проверочного датасета и посмотрим на расстояния для позитивных и негативных пар. Если модель обучилась, расстояния для позитывных пар будут меньше.

P.S. По умолчанию TripletLoss минимизирует Евклидово расстояние.

In [ ]:
import torchvision
import torch.nn.functional as F
import numpy as np

# Helper method for visualization
def show(img, text=None):
    img_np = img.numpy()
    plt.axis("off")
    plt.text(75, 120, text, fontweight="bold")
    plt.imshow(np.transpose(img_np, (1, 2, 0)))
    plt.show()

def plot_imgs(model,test_loader):
    distances_pos = []
    distances_neg = []
    model.eval()
    with torch.inference_mode():
      for i, batch in enumerate(test_loader, 0):
        anc, pos, neg = batch
        output_anc, output_pos, output_neg = model(anc.to(device), pos.to(device), neg.to(device))
        # compute euc. distance
        distance_pos = F.pairwise_distance(output_anc, output_pos).item() 
        distance_neg = F.pairwise_distance(output_anc, output_neg).item() 

        distances_pos.append(distance_pos)
        distances_neg.append(distance_neg)

        if not i % 5 :
                concatenated = torch.cat((anc, pos, neg))
                show(    
                    torchvision.utils.make_grid(concatenated),
                    f"Positive / negative euclidean distances: {distance_pos:.3f} / {distance_neg:.3f}",
                )

    return distances_pos, distances_neg


distances_pos, distances_neg = plot_imgs(model,val_loader)


Но такая оценка субъективна, давайте посмотрим на распределение расстояний по категориям:

In [ ]:
import seaborn as sns

distnaces = {"The same person": distances_pos, "Another person": distances_neg}

ax = sns.displot(distnaces, kde=True, stat="density")
ax.set(xlabel="Pairwise distance")
plt.show()

Видно что для фото одного и того же человека, в большинстве случаев расстояние лежит в интервале от 0 до 5.5. 

А вот для фото разных людей от 5.5 до 19. 

Если бы мы проектировали систему распознавания лиц, нужно было бы выбрать порог что бы сравнивать с ним расстояние и принимать решение о том верифицировать фото как подлинное или нет.

Соответственно, для нашего игрушечного датасета такой порог следует выбирать ~= 6 при условии что ошибки первого и второго рада для нас равнозначны.

Ссылки по теме:

[Ищем знакомые лица](https://habr.com/ru/post/317798/)

https://www.sciencedirect.com/science/article/pii/S2351978920300263?via%3Dihub

## Few-shots learning in GPT

Есть и другой интересный вариант, как можно использовать few-shots learning. Огромные модели (такие как GPT-3 и варианты) в целом способны генерировать любой контент. Вопрос лишь в том, как бы им объяснить что мы от них хотим.

[Language Models are Few-Shot Learners (Brown et al., 2020)](https://arxiv.org/abs/2005.14165)

GPT-3 не доступна для свободного пользования (OpenAI оказался не очень-то и open). Но, умельцы из сообщества ElutherAI сделали open-source версию **GPT-J**, которая работает сравнимо с оригинальной моделью. Более того, они ее захостили у себя на сайте, что бы каждый мог пользоваться без долгой и муторной подгрузки модели (установка зависимостей и загрузка весов для GPT-J занимает ~20 минут в колабе). Ей-то мы и воспользуемся.

Зайдем на [сайт модели](https://6b.eleuther.ai/)

Перед нами текстовый редактор, в который можно написать что угодно. Для примера давайте попробуем перефразирововать предложение: 

    Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task.

Например сформулировав задачу так: *paraphrase* `...sentence...`

Что-то результаты не впечатляют. Тут-то на помощь и приходит идея Few-shots. 

Что если мы в качестве затравки (*prompt*) скормим моделе 2-3 примера в которых покажим чего от нее хотим?

Давайте попробуем вот такой prompt:

    Original: Her life spanned years of incredible change for women as they gained more rights than ever before.
    Paraphrase: She lived through the exciting era of women's liberation.

    Original: Giraffes like Acacia leaves and hay, and they can consume 75 pounds of food a day.
    Paraphrase: A giraffe can eat up to 75 pounds of Acacia leaves and hay daily.

    Original: Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task.
    Paraphrase:

Значительно лучше! Ответ который получил автор блокнота с первого раза: 

**Paraphrase:** *Recent work has shown that you can get a lot of money for a lot of text if you pre-train it for a specific task, then fine-tune it.*

Теперь давайте попробуем с числовыми рядами.

    Sequence: 2, 4,  6, 8
    Continuation: 16, 32, 64, 128

    Sequence: 3, 9, 27, 81
    Continuation: 243, 729, 2187, 6561

    Sequence: 4, 16, 64, 256
    Continuation: 

И с геологической викториной:

    Q: The molten rock that sits below the earth's surface is called what?
    A: Magma

    Q: Diamonds rank at number 10 on the Mohs scale, meaning they are the hardest mineral listed. Where would cubic zirconia (CZ) rank on the scale?
    A: 8-8.5

    Q: The emerald is a type of beryl. The distinctive green color of the emerald derives from trace amounts of what metallic elements?
    A: 

Ответы лучше перепроверить, а вот вопросы для викторины готовы.

Не стесняйтесь придумывать свои собственные примеры и задачи! Have fun

## Оптимизация гиперпараметров

Часто, когда мы пишем и обучаем сети (будь то с нуля или с помощью transfer learning) мы вынуждены угадывать гиперпараметры (lr, betas и тд). В случае с learning rate нам есть от чего оттолкнуться (маленький lr для transfer learning), [константа Karpathy (3e-4)](https://twitter.com/karpathy/status/801621764144971776?lang=en) для Adam, но все же, такой подход не кажется оптимальным.

Для оптимизации гиперпараметров существуют готовые решения, которые используют различные методы black-box оптимизации. Разберем одну из наиболее популярных библиотек - [**Optuna**](https://optuna.org/)

In [ ]:
!pip install --quiet optuna

Давайте оптимизируем наш learning rate

In [ ]:
import torch
import optuna

# define function which will optimized
def objective(trial):
    # boundaries for the optimizer's 
    lr = trial.suggest_loguniform("lr", 1e-6, 1e-2)
    
    ##### If you need more parameters for optimization, it is done like this:
    #new_parameter =  trial.suggest_loguniform("new_parameter", lower_bound, upper_bound)

    # create new model(and all parameters) every iteration
    model = SiameseNet().to(device)
    criterion = nn.TripletMarginLoss()
    optimizer = optim.Adam(
        model.parameters(), lr=lr
    )  # learning step regulates by optuna
    

    # To save time, we will take only 3 epochs
    _, last_epoch_loss = train(3, model, criterion, optimizer, train_loader)
    return last_epoch_loss


# Create "exploration"
study = optuna.create_study(direction="minimize", study_name="Optimal lr")

#
study.optimize(
    objective, n_trials=10
)  # The more iterations, the higher the chances of catching the most optimal hyperparameters

In [ ]:
# show best params
study.best_params

Конечно же, можно оптимизировать сразу несколько параметров за раз, а еще в качестве параметров можеть выступать сама архитектура сети (например количесвто слоев и каналов). Это можно легко сделать по анологии.

Давайте посмотрим на историю оптимизации нашего lr

In [ ]:
optuna.visualization.plot_optimization_history(study)

Что ж, проверим а станет ли реально лучше

In [ ]:
model = SiameseNet().to(device)
criterion = nn.TripletMarginLoss()
optimizer = optim.Adam(
    model.parameters(), lr=study.best_params["lr"]
)  # take lr, which choosen Optuna
#scheduler = ReduceLROnPlateau(optimizer, "min", verbose=True, factor=0.5, patience=3)

l_optim, _ = train(5,  model, criterion, optimizer, train_loader)

In [ ]:
plt.plot([i for i in range(len(l))], l, label="no optimization")
plt.plot([i for i in range(len(l))], l_optim[:len(l)], label="optimal params")
plt.ylabel('loss')
plt.xlabel('num of epochs')
plt.grid()
plt.legend()
plt.show()

In [ ]:

distances_pos, distances_neg = plot_imgs(model,val_loader)

In [ ]:
import seaborn as sns

distnaces_optim = {"The same person": distances_pos, "Another person": distances_neg}

ax_0 = sns.displot(distnaces, kde=True, stat="density", alpha=0.5)
ax_1 = sns.displot(distnaces_optim, kde=True, stat="density", alpha=0.5)
ax_0.set(xlabel="Pairwise distance")
plt.show()

<font size = "6"> Заключение


Мы затронули проблемы, которые возникают при обучени на реальных данных.

Одна из основных проблем - малые датасеты. Для того, чтобы обучить нейронку на небольшом датасете можно использовать:
- `аугментацию`
- `Tranfer learning`
Однако необходимо помнить, что ни один из этих методов не защитит от ситуации, когда реальные данные будут сильно отличаться от тренировочных.

В случае, когда у нас не только мало данных, но и еще и очень большое (возможно, неизвестное) число классов, можно воспользоваться `One-shot Learning`. В этом случае нейронка обучается не классифицировать изображения, а, наоборот, находить различия между классом и новыми данными. Для этого используются нейронные сети, относящиеся к классу сиамских нейронных сетей.

Так же мы разобрали автоматическую оптимизацию гиперпараметров с помощью Optuna и показали, что это эффективный способ сделать нейросеть лучше малой ценой.

<font size = "6">Литература


<font size = "5"> Обучение на реальных данных

[How to avoid machine learning pitfalls: a guide for academic researchers (Lones, 2021)](https://arxiv.org/abs/2108.02497)

[Understanding data augmentation for classification: when to warp? (Wong et al., 2016)](https://arxiv.org/abs/1609.08764) 

[Learning from class-imbalanced data: Review of methods and applications (Haixiang et al., 2017)](https://www.sciencedirect.com/science/article/abs/pii/S0957417416307175?via%3Dihub)

<font size = "5"> Как решить проблему маленького количества данных?

[Блог-пост о том как решать проблему малого количества данных.](https://towardsdatascience.com/breaking-the-curse-of-small-datasets-in-machine-learning-part-1-36f28b0c044d)

<font size = "5"> Несбалансированные данные

[Imbalanced Data: How to handle Imbalanced Classification Problems](https://www.analyticsvidhya.com/blog/2017/03/imbalanced-data-classification/).

[SMOTE explained for noobs - Synthetic Minority Over-sampling TEchnique line by line](https://rikunert.com/SMOTE_explained)

[Блог пост про 8 тактик борьбы с несбалансированными классами в наборе данных машинного обучения](https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/)

[Метрики разработаные для работы с несбалансированными классами.](https://machinelearningmastery.com/classification-accuracy-is-not-enough-more-performance-measures-you-can-use/#:~:text=Classification%20Accuracy%20is%20Not%20Enough%3A%20More%20Performance%20Measures%20You%20Can%20Use,-By%20Jason%20Brownlee&text=When%20you%20build%20a%20model,This%20is%20the%20classification%20accuracy)

[Творческий подход](https://www.quora.com/In-classification-how-do-you-handle-an-unbalanced-training-set)

<font size = "5"> Transfer Learning

[Image Classification using Transfer Learning in Pytorch](https://learnopencv.com/image-classification-using-transfer-learning-in-pytorch/)

[How To Do Transfer Learning For Computer Vision | PyTorch Tutorial](https://www.youtube.com/watch?v=6nQlxJvcTr0)

[Transfer learning for Computer Vision Tutorial](https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html)

[Python Pytorch Tutorials # 2 Transfer Learning : Inference with ImageNet Models](https://www.youtube.com/watch?v=Upw4RaERZic)

[PyTorch - The Basics of Transfer Learning with TorchVision and AlexNet](https://www.youtube.com/watch?v=8etkVC93yU4)


<font size = "5">Augmentation

[A survey on Image Data Augmentation for Deep Learning (Shorten and Khoshgoftaar, 2019)](https://journalofbigdata.springeropen.com/articles/10.1186/s40537-019-0197-0)

[Data augmentation for improving deep learning in image classification problem](https://www.researchgate.net/publication/325920702_Data_augmentation_for_improving_deep_learning_in_image_classification_problem)


<font size = "5"> Few-shot learning

[One-Shot Learning with Siamese Networks using Keras](https://towardsdatascience.com/one-shot-learning-with-siamese-networks-using-keras-17f34e75bb3d)

[One-Shot image classification by meta learning](https://medium.com/nerd-for-tech/one-shot-learning-fe1087533585)

[One-Shot Learning (Part 1/2): Definitions and fundamental techniques](https://heartbeat.fritz.ai/one-shot-learning-part-1-2-definitions-and-fundamental-techniques-1df944e5836a)

[One-Shot Learning (Part 2/2): Facial Recognition Using a Siamese Network](https://heartbeat.fritz.ai/one-shot-learning-part-2-2-facial-recognition-using-a-siamese-network-5aee53196255)

[FaceNet: A Unified Embedding for Face Recognition and Clustering (Schroff et al., 2015)](https://arxiv.org/abs/1503.03832)

[One-Shot Learning explained using FaceNet](https://medium.com/intro-to-artificial-intelligence/one-shot-learning-explained-using-facenet-dff5ad52bd38)

[Ищем знакомые лица](https://habr.com/ru/post/317798/)

[Siamese Neural Networks for One-shot Image Recognition (Koch et al., 2015)](https://www.cs.cmu.edu/~rsalakhu/papers/oneshot1.pdf)

[Dimensionality Reduction by Learning an Invariant Mapping (Hadsell et al., 2005)](http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf)

[Language Models are Few-Shot Learners (Brown et al., 2020)](https://arxiv.org/abs/2005.14165)

<font size = "5"> Hyperparameter optimization

[Tuning Hyperparameters with Optuna](https://towardsdatascience.com/tuning-hyperparameters-with-optuna-af342facc549)